In [2]:
using Pkg
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("Distances")
Pkg.add("Distributions")
Pkg.add("DataFrames")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


...

  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [3]:
using JuMP
using Gurobi
using DataFrames
using Random
# stochastic departure: known when it comes in, but doesn't know when it departs (stoch demand)
# first (where to store), second ()

# Generate Data

## Constants

In [4]:
warehouse_location_side_size = 2
warehouse_rows = 20
warehouse_columns = 50

num_shipments = 100
start_time = 5
time_window = 20
time_horizon = 30
max_item_size = 10

S = num_shipments
L = warehouse_rows * warehouse_columns
end_time = start_time + time_window
T = time_horizon
#0|----|s_t=5  e_t=25|----|30
# indicator_arr2loc = zeros(Int, S)
# indicator_loc2dep  = zeros(Int, S)
# for s in 1:S
#     if start_time -1 < shipments[s].arrival_time
#         indicator_arr2loc[s] = 1
#     end
#     if shipments[s].departure_time < end_time
#         indicator_loc2dep[s] = 1
#     end
# end
# adding x, change obj
# @variable(
#     model,
#     r[1:S,1:L,1:T] >= 0, Bin
#     x[1:S,1:L] >= 0, Bin
# ) ;

# @objective(
#     model,
#     Min,
#     sum(
#         distance_to_arrival[l] * indicator_arr2loc[s] * x[s,l] + distance_to_departure[l] * indicator_loc2dep[s] * x[s,l]
#         for s in 1:S,
#             l in 1:L
#     )
# );

# #x[s, l] shipment stored in location l
# #r[s,l,t] -> r[s,l] #at a give time and location only one product can be stored 
# for s in 1:S
#     for l in 1:L
#             @constraint(
#                 x[s, l] = sum(r[s,l,t] for l in 1:T) > 0
#             )
#     end
# end

30

In [5]:
shipments

UndefVarError: UndefVarError: shipments not defined

In [6]:
arrival_location = (0, 1)
departure_location = (warehouse_rows, 0)

distance_to_arrival = zeros(warehouse_rows * warehouse_columns)
for i in 1:warehouse_rows, j in 1:warehouse_columns
    l = (i-1)*warehouse_columns + j
    distance_to_arrival[l] = i * warehouse_location_side_size + j * warehouse_location_side_size
end

distance_to_departure = zeros(warehouse_rows * warehouse_columns)
for i in 1:warehouse_rows, j in 1:warehouse_columns
    l = (i-1)*warehouse_columns + j
    distance_to_departure[l] = ((warehouse_rows - i) * warehouse_location_side_size) + j * warehouse_location_side_size
end

location_max_size = rand(1:max_item_size, warehouse_rows * warehouse_columns)

Random.seed!(3000)
rand_shipment_data = rand(1:T, (S, 2))
arrivals = minimum(rand_shipment_data, dims=2)
departures = maximum(rand_shipment_data, dims=2)
# TODO there cannot be any only arrival or only departures
rand_sizes = rand(1:max_item_size, S)

struct Shipment
    arrival_time::Int
    departure_time::Int # probability vector 
    product_size::Int
end

shipments = Array{Shipment,1}(undef, S)

for s in 1:S
    arrival_time = arrivals[s]
    departure_time = departures[s]
    product_size = rand_sizes[s]

    # Can't arrive and depart at the same time
    if arrival_time == departure_time
        if departure_time == T
            arrival_time -= 1
        else
            departure_time += 1
        end
    end

    shipments[s] = Shipment(arrival_time, departure_time, product_size)
end

# Build Model

In [7]:
shipments

100-element Vector{Shipment}:
 Shipment(9, 17, 1)
 Shipment(15, 21, 9)
 Shipment(3, 22, 2)
 Shipment(3, 16, 1)
 Shipment(7, 19, 3)
 Shipment(21, 27, 6)
 Shipment(13, 25, 1)
 Shipment(14, 18, 4)
 Shipment(1, 18, 10)
 Shipment(11, 27, 6)
 ⋮
 Shipment(2, 3, 3)
 Shipment(15, 25, 8)
 Shipment(11, 19, 10)
 Shipment(8, 25, 4)
 Shipment(11, 15, 6)
 Shipment(2, 3, 10)
 Shipment(11, 25, 4)
 Shipment(8, 26, 3)
 Shipment(9, 26, 2)

In [ ]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 60);

# Are we assigning the product from shipment s to location l at time t
@variable(
    model,
    r[1:S,1:L,1:T] >= 0, Bin
    x[1:S,1:L] >= 0, Bin
) ;

@objective(
    model,
    Min,
    sum(
        distance_to_arrival[l] * indicator_arr2loc[s] * x[s,l] + distance_to_departure[l] * indicator_loc2dep[s] * x[s,l]
        for s in 1:S,
            l in 1:L
    )
);

#x[s, l] shipment stored in location l
#r[s,l,t] -> r[s,l] #at a give time and location only one product can be stored 
for s in 1:S
    for l in 1:L
            @constraint(
                x[s, l] = sum(r[s,l,t] for l in 1:T) > 0
            )
    end
end

# Each product must be assigned to exactly one location during its time window:
for s in 1:S
    for t in shipments[s].arrival_time:shipments[s].departure_time
        @constraint(
            model,
            sum(r[s,l,t] for l in 1:L) == 1
        );
    end
end

# Each location can hold at most one product at any specific time:
for l in 1:L
    for t in 1:T
        @constraint(
            model,
            sum(r[s,l,t] for s in 1:S) <= 1
        );
    end
end

# Each location can only hold items up to max size of the location
for l in 1:L
    for t in 1:T
        for s in 1:S
            @constraint(
                model,
                r[s,l,t] * shipments[s].product_size <= location_max_size[l]
            );
        end
    end
end

In [ ]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 60);

# Are we assigning the product from shipment s to location l at time t
@variable(
    model,
    r[1:S,1:L,1:T] >= 0, Bin
) ;

@objective(
    model,
    Min,
    sum(
        (distance_to_arrival[l] + distance_to_departure[l]) / (shipments[s].departure_time - shipments[s].arrival_time) * r[s,l,t]
        for s in 1:S,
            l in 1:L,
            t in 1:T
    )
);

# Each product must be assigned to exactly one location during its time window:
for s in 1:S
    for t in shipments[s].arrival_time:shipments[s].departure_time
        @constraint(
            model,
            sum(
                r[s,l,t]
                for l in 1:L
            ) == 1
        );
    end
end

# Each product is only assigned to a location during its time window:
for s in 1:S
    for t in 1:shipments[s].arrival_time-1
        @constraint(
            model,
            sum(
                r[s,l,t]
                for l in 1:L
            ) == 0
        );
    end
end
for s in 1:S
    for t in shipments[s].departure_time+1:T
        @constraint(
            model,
            sum(
                r[s,l,t]
                for l in 1:L
            ) == 0
        );
    end
end

# Each location can hold at most one product at any specific time:
for l in 1:L
    for t in 1:T
        @constraint(
            model,
            sum(
                r[s,l,t]
                for s in 1:S
            ) <= 1
        );
    end
end

# Each location can only hold items up to max size of the location
for l in 1:L
    for t in 1:T
        for s in 1:S
            @constraint(
                model,
                r[s,l,t] * shipments[s].product_size <= location_max_size[l]
            );
        end
    end
end

In [33]:
optimize!(model) 
# before: 
# Optimize a model with 3033000 rows, 3000000 columns and 9000000 nonzeros
# Model fingerprint: 0x1cff127a
# Variable types: 0 continuous, 3000000 integer (3000000 binary)
# Coefficient statistics:
#   Matrix range     [1e+00, 1e+01]
#   Objective range  [2e+00, 2e+02]
#   Bounds range     [0e+00, 0e+00]
#   RHS range        [1e+00, 1e+01]
# Found heuristic solution: objective 17061.186114
# Presolve removed 3003182 rows and 2376530 columns (presolve time = 5s) ...
# Presolve removed 3031945 rows and 2968090 columns
# Presolve time: 6.79s
# Presolved: 1055 rows, 31910 columns, 63820 nonzeros
# Found heuristic solution: objective 5798.9611481
# Variable types: 0 continuous, 31910 integer (31910 binary)
# Found heuristic solution: objective 5736.0499637

Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3033000 rows, 3000000 columns and 9000000 nonzeros
Model fingerprint: 0x1cff127a
Variable types: 0 continuous, 3000000 integer (3000000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 17061.186114
Presolve removed 3003182 rows and 2376530 columns (presolve time = 5s) ...
Presolve removed 3031945 rows and 2968090 columns
Presolve time: 6.79s
Presolved: 1055 rows, 31910 columns, 63820 nonzeros
Found heuristic solution: objective 5798.9611481
Variable types: 0 continuous, 31910 integer (31910 binary)
Found heuristic solution: objective 5736.0499637

Root simplex log...

Iteration    Objective       Primal Inf.    D